### Main body of code, generates wavepacket data

In [24]:
from IPython.display import HTML
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import numpy.random as random
import os
import pandas as pd
from scipy import linalg as ln
from scipy import sparse as sparse

class Wave_Packet:
    def __init__(self, epsilon, spacing, V_type, seed, dt = 0.25, x_range = 40, no_steps = 300, sigma0 = 1.5, k0 = 5.5):
        # instantiating constants
        self.N = no_steps
        self.epsilon = epsilon
        self.spacing = spacing
        self.x, self.dx = np.linspace(-1*x_range/2, x_range/2, self.N, retstep = True)
        self.dt = dt
        self.seed = seed
        
        # generation of Gaussian wavepacket 
        norm = (2.0 * np.pi * sigma0**2)**(-0.25)
        self.psi = np.exp(-(self.x - 0)**2 / (4.0 * sigma0**2))
        self.psi = self.psi*np.exp(1.0j * k0 * self.x)
        self.psi *= norm
        
        ## psi for perturbed
        self.psi_perturbed = self.psi
        
        # generation of the Hamiltonian matrix and introduction of disorder
        V = np.zeros(self.N)
        self.V = V
        # free particle
        if V_type == 'Free':
            pass
        # QHO
        elif V_type == 'QHO':
            for i in range(50,250):
                V[i] = self.x[i]**2 / self.N
#         # morse potential
#         elif V_typle == 'Morse':
#             pass 
        
        # disorder with set seed
        perturbation = epsilon*random.uniform(-1, 1, size = self.N)
        # creating desired spacin in disorder
        disorder = np.zeros(self.N)
        disorder[::self.spacing + 1] = perturbation[::self.spacing + 1]
        
        self.V_perturbed = self.V + disorder
    
        # arrays for diagonal and off-diagonal values in Hamiltonian
        diag = np.zeros(self.N)
        offdiag = np.zeros(self.N)
        I = np.identity(self.N)
        
        ## normal wavepacket
        # creating the Hamiltonian for unperturbed wavepacket
        for i in range(self.N):
            diag[i] = 1 + V[i]
        for i in range(self.N):
            offdiag[i] = - 0.5
        H = np.zeros(shape=(self.N, self.N))
        for j in range(self.N):
            per = (j+1) % self.N  # periodic
            H[j, j] = diag[j]
            H[j, per] = offdiag[j]
            H[per, j] = offdiag[j]
        hamiltonian = H
        
        # Crank-Nicolson method to calculate time evolution of normal wavepacket
        backward = (I - self.dt / 2.0j * hamiltonian)
        forward = (I + self.dt / 2.0j * hamiltonian)
        self.evolution_matrix = ln.inv(backward).dot(forward)

        ## perturbed wavepacket
        # creating the Hamiltonian for perturbed wavepacket
        for i in range(self.N):
            diag[i] = 1 + V[i] + disorder[i]
        for i in range(self.N):
            offdiag[i] = - 0.5
        H = np.zeros(shape=(self.N, self.N))
        for j in range(self.N):
            per = (j+1) % self.N  # periodic
            H[j, j] = diag[j]
            H[j, per] = offdiag[j]
            H[per, j] = offdiag[j]
        hamiltonian_perturbed = H
        
        # Crank-Nicolson method to calculate time evolution of perturbed wavepacket
        
        backward = (I - self.dt / 2.0j * hamiltonian_perturbed)
        forward = (I + self.dt / 2.0j * hamiltonian_perturbed)
        self.evolution_matrix_perturbed = ln.inv(backward).dot(forward)        
      
    def evolve(self):
        ## normal wavepacket
        # evolves wave packet
        self.psi = self.evolution_matrix.dot(self.psi)
        # probability density of wave
        self.prob = np.real(self.psi * np.conjugate(self.psi))
        
        norm = sum(self.prob) # normalises to begin with, then should be 1
        self.prob /= norm
        self.psi /= norm**0.5
        
        ## perturbed wavepacket
        # evolves wave packet
        self.psi_perturbed = self.evolution_matrix_perturbed.dot(self.psi_perturbed)
        # probability density of wave
        self.prob_perturbed = np.real(self.psi_perturbed * np.conjugate(self.psi_perturbed))
        
        norm_perturbed = sum(self.prob_perturbed) # normalises to begin with, then should be 1
        self.prob_perturbed /= norm_perturbed
        self.psi_perturbed /= norm_perturbed**0.5
        
        ## inner product between psi and perturbed psi
        inner = abs(self.psi * np.conjugate(self.psi_perturbed))
        self.norm_inner = sum(inner)
        
        return self.prob, self.prob_perturbed, self.norm_inner

class Evolution_Generator:
    def __init__(self, wave_packet, max_steps = 10):
        self.wave_packet = wave_packet
        self.max_steps = max_steps
        self.wave_data = []
        self.inner_data = []
        
    def data_output(self):    
        for i in range(self.max_steps):
            self.wave_data.append(self.wave_packet.evolve())
            self.inner_data.append(self.wave_packet.evolve()[2])
       
        return self.wave_data, self.inner_data

### ANIMATION (FOR VERIFICATION)

In [26]:
epsilon = 3.0
spacing = 0 
V_type = 'QHO'
seed = 1

wavep = Wave_Packet(epsilon, spacing, V_type, seed)
wave_data = Evolution_Generator(Wave_Packet(epsilon, spacing, V_type, seed)).data_output()[0]

fig,ax1=plt.subplots()

axtext = fig.add_axes([0.0,0.95,0.1,0.05])
axtext.axis("off")

time = axtext.text(0.5, 0.5, str(0), ha="left", va="top")

psi, = ax1.plot([],[])
psi_perturbed, = ax1.plot([],[])


def animate(j):
    ax1.clear()
    ax1.set_title('Time evolution of perturbed and unperturbed wavepackets in {}'.format(V_type))
    ax1.set_xlabel('Position, a$_0$')
    ax1.set_ylabel('Probability density, $|Ψ(x)|^2$')
    ax1.set_ylim([0,0.3])
    ax1.plot(Wave_Packet(epsilon, spacing, V_type, seed).x, Wave_Packet(epsilon, spacing, V_type, seed).V/5, color = 'b', label = 'Potential')
    ax1.fill_between(Wave_Packet(epsilon, spacing, V_type, seed).x, Wave_Packet(epsilon, spacing, V_type, seed).V/5, facecolor="none", hatch="/", edgecolor="b", linewidth=0.0)
    ax1.plot(Wave_Packet(epsilon, spacing, V_type, seed).x, wave_data[j][0], label = 'Wavepacket (unperturbed)')
    ax1.plot(Wave_Packet(epsilon, spacing, V_type, seed).x, wave_data[j][1], label = 'Wavepacket (perturbed, ε = {})'.format(epsilon))
    ax1.legend()
    psi.set_data(Wave_Packet(epsilon, spacing, V_type, seed).x, wave_data[j][0])
    psi_perturbed.set_data(Wave_Packet(epsilon, spacing, V_type, seed).x, wave_data[j][1])
    
    time.set_text(('Elapsed time: {:6.2f} fs').format(j * Wave_Packet(epsilon, spacing, V_type, seed).dt * 2.419e-2))
    
    return psi, psi_perturbed, time,

wave_animation = animation.FuncAnimation(fig, animate, frames = len(wave_data), interval = 50)
HTML(wave_animation.to_html5_video())
#wave_animation.save(r'C:\Users\emgsh\OneDrive\Documents\Stuff\Important Documents\University\Maths\Quantum Tunneling Project\test.mp4', 
#          writer = 'ffmpeg', fps = 30)

RuntimeError: Requested MovieWriter (ffmpeg) not available

### Dumps data files to data analysis folder

In [10]:
V_type = 'QHO'
path = r'D:\Quantum Tunneling Project\Code\data_for_analysis\100'

def data_toexport(epsilonvals, spacingvals, V_type, seeds, path, iter_over, fresh): 
    ## ensuring that it iterates over multiple runs of one variable 
    if iter_over == 'epsilon':
        epsilonvals = [epsilonvals]
        runstr = 'e_{}'.format(str(epsilonvals[0]))
    
    elif iter_over  == 'spacing':
        spacingvals = [spacingvals]
        runstr = 's_{}'.format(str(spacingvals[0]))
        
    ## if fresh data required, clears containing folder
    if fresh == True:
        if input('This will delete files, are you sure that you want to continue (Y/N)? [ENSURE PATH IS CORRECT] ').lower() == 'y':
            print('CURRENT RUN: {}'.format(runstr))
            for folder in os.listdir(path):
                for subfolder in os.listdir(path + '\\' + folder):
                    if subfolder.startswith('epsilon') or subfolder.startswith('spacing'):
                        for file in os.listdir(path + '\\' + folder + '\\' + subfolder):
                            os.remove(path + '\\' + folder + '\\' + subfolder + '\\' + file)
                        os.rmdir(path + '\\' + folder + '\\' + subfolder)
                    elif subfolder.endswith('.csv'):
                        os.remove(path + '\\' + folder + '\\' + subfolder)
                        
                os.rmdir(path + '\\' + folder)
        else:
            return 'ABORTING PROGRAMME'
    else:
        print('CURRENT RUN: {}'.format(runstr))
    
    ## system parameters  
    seeds = seeds
    newdir1 = path + '\\' + 'QTP_SYSTEM_RUN_{}'.format(str(runstr))
    os.mkdir(newdir1)
    param_init = Wave_Packet(0, 0, V_type, 0)
    dt = param_init.dt
    no_steps = len(Evolution_Generator(param_init).data_output()[1])
    total_runtime = dt * no_steps * 2.419e-2
    sys_params = newdir1 + '\\' + 'sys_params.csv'

    if iter_over == 'epsilon':
        epsilon = epsilonvals[0]
        for spacing in spacingvals:
            print('--- SPACING: {}'.format(spacing))
            newdir2 = newdir1 + '\\' + 'spacing_{}'.format(str(spacing))
            os.mkdir(newdir2)
            for seed in seeds:
                wavegen = Wave_Packet(epsilon, spacing, V_type, seed)
                inner_data = Evolution_Generator(wavegen).data_output()[1]
                print('------ SEED: {}'.format(seed))
                np.random.seed(seed)

                # write parameters to file
                parameters = {'epsilonvals': epsilonvals, 'spacingvals': spacingvals, 'dt': dt,
                              'no_steps': no_steps, 'total_runtime': total_runtime, 'iter_over': iter_over}
                df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in parameters.items()]))
                df.to_csv(sys_params, index = False)
                data_file = newdir2 + '\\' + 'SEED_{}.txt'.format(str(seed))

                #write data to file
                with open(data_file, 'x') as data:
                    for i in inner_data:
                        data.write(str(i) + ' ')    

        return

    elif iter_over == 'spacing':
        spacing = spacingvals[0]
        for epsilon in epsilonvals:
            print('--- EPSILON: {}'.format(epsilon))
            newdir2 = newdir1 + '\\' + 'epsilon_{}'.format(str(epsilon))
            os.mkdir(newdir2)
            for seed in seeds:
                # setting data to generate and choosing save location
                wavegen = Wave_Packet(epsilon, spacing, V_type, seed)
                inner_data = Evolution_Generator(wavegen).data_output()[1]
                print('------ SEED: {}'.format(seed))
                np.random.seed(seed)

                # write parameters to file
                parameters = {'epsilonvals': epsilonvals, 'spacingvals': spacingvals, 'dt': dt,
                              'no_steps': no_steps, 'total_runtime': total_runtime, 'iter_over': iter_over}
                df = pd.DataFrame(dict([(k,pd.Series(v)) for k,v in parameters.items()]))
                df.to_csv(sys_params, index = False)
                data_file = newdir2 + '\\' + 'SEED_{}.txt'.format(str(seed))

                #write data to file
                with open(data_file, 'x') as data:
                    for i in inner_data:
                        data.write(str(i) + ' ')

        return

### Iterator, iterates over whatever value you like as well as over seeds

In [ ]:
epsilonvals = []
spacingvals = []
for i in range(21):
    epsilonvals.append(i/4)
    
for i in range(21):
    spacingvals.append(i*10)
    
print(epsilonvals, spacingvals)

seeds = np.arange(100)

def iterator(value, seeds):
    i = 0
    if value == 'epsilon':
        for epsilon in epsilonvals:
            if i == 0:
                data_toexport(epsilon, spacingvals, V_type, seeds, path, iter_over = value, fresh = True)
                i = 1
            elif i == 1:
                data_toexport(epsilon, spacingvals, V_type, seeds, path, iter_over = value, fresh = False)
                
    
    elif value == 'spacing':
        for spacing in spacingvals:
            if i == 0:
                data_toexport(epsilonvals, spacing, V_type, seeds, path, iter_over = value, fresh = True)
                i = 1
            elif i == 1:
                data_toexport(epsilonvals, spacing, V_type, seeds, path, iter_over = value, fresh = False)
        print('COMPLETE')
    return

iterator('spacing', seeds)

[0.0, 0.25, 0.5, 0.75, 1.0, 1.25, 1.5, 1.75, 2.0, 2.25, 2.5, 2.75, 3.0, 3.25, 3.5, 3.75, 4.0, 4.25, 4.5, 4.75, 5.0] [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200]
This will delete files, are you sure that you want to continue (Y/N)? [ENSURE PATH IS CORRECT] y
CURRENT RUN: s_0
--- EPSILON: 0.0
------ SEED: 0
------ SEED: 1
------ SEED: 2
------ SEED: 3
------ SEED: 4
------ SEED: 5
------ SEED: 6
------ SEED: 7
------ SEED: 8
------ SEED: 9
------ SEED: 10
------ SEED: 11
------ SEED: 12
------ SEED: 13
------ SEED: 14
------ SEED: 15
------ SEED: 16
------ SEED: 17
------ SEED: 18
------ SEED: 19
------ SEED: 20
------ SEED: 21
------ SEED: 22
------ SEED: 23
------ SEED: 24
------ SEED: 25
------ SEED: 26
------ SEED: 27
------ SEED: 28
------ SEED: 29
------ SEED: 30
------ SEED: 31
------ SEED: 32
------ SEED: 33
------ SEED: 34
------ SEED: 35
------ SEED: 36
------ SEED: 37
------ SEED: 38
------ SEED: 39
------ SEED: 40
------ SEED:

------ SEED: 90
------ SEED: 91
------ SEED: 92
------ SEED: 93
------ SEED: 94
------ SEED: 95
------ SEED: 96
------ SEED: 97
------ SEED: 98
------ SEED: 99
--- EPSILON: 1.25
------ SEED: 0
------ SEED: 1
------ SEED: 2
------ SEED: 3
------ SEED: 4
------ SEED: 5
------ SEED: 6
------ SEED: 7
------ SEED: 8
------ SEED: 9
------ SEED: 10
------ SEED: 11
------ SEED: 12
------ SEED: 13
------ SEED: 14
------ SEED: 15
------ SEED: 16
------ SEED: 17
------ SEED: 18
------ SEED: 19
------ SEED: 20
------ SEED: 21
------ SEED: 22
------ SEED: 23
------ SEED: 24
------ SEED: 25
------ SEED: 26
------ SEED: 27
------ SEED: 28
------ SEED: 29
------ SEED: 30
------ SEED: 31
------ SEED: 32
------ SEED: 33
------ SEED: 34
------ SEED: 35
------ SEED: 36
------ SEED: 37
------ SEED: 38
------ SEED: 39
------ SEED: 40
------ SEED: 41
------ SEED: 42
------ SEED: 43
------ SEED: 44
------ SEED: 45
------ SEED: 46
------ SEED: 47
------ SEED: 48
------ SEED: 49
------ SEED: 50
------ SEED: 51


------ SEED: 0
------ SEED: 1
------ SEED: 2
------ SEED: 3
------ SEED: 4
------ SEED: 5
------ SEED: 6
------ SEED: 7
------ SEED: 8
------ SEED: 9
------ SEED: 10
------ SEED: 11
------ SEED: 12
------ SEED: 13
------ SEED: 14
------ SEED: 15
------ SEED: 16
------ SEED: 17
------ SEED: 18
------ SEED: 19
------ SEED: 20
------ SEED: 21
------ SEED: 22
------ SEED: 23
------ SEED: 24
------ SEED: 25
------ SEED: 26
------ SEED: 27
------ SEED: 28
------ SEED: 29
------ SEED: 30
------ SEED: 31
------ SEED: 32
------ SEED: 33
------ SEED: 34
------ SEED: 35
------ SEED: 36
------ SEED: 37
------ SEED: 38
------ SEED: 39
------ SEED: 40
------ SEED: 41
------ SEED: 42
------ SEED: 43
------ SEED: 44
------ SEED: 45
------ SEED: 46
------ SEED: 47
------ SEED: 48
------ SEED: 49
------ SEED: 50
------ SEED: 51
------ SEED: 52
------ SEED: 53
------ SEED: 54
------ SEED: 55
------ SEED: 56
------ SEED: 57
------ SEED: 58
------ SEED: 59
------ SEED: 60
------ SEED: 61
------ SEED: 62
--

------ SEED: 11
------ SEED: 12
------ SEED: 13
------ SEED: 14
------ SEED: 15
------ SEED: 16
------ SEED: 17
------ SEED: 18
------ SEED: 19
------ SEED: 20
------ SEED: 21
------ SEED: 22
------ SEED: 23
------ SEED: 24
------ SEED: 25
------ SEED: 26
------ SEED: 27
------ SEED: 28
------ SEED: 29
------ SEED: 30
------ SEED: 31
------ SEED: 32
------ SEED: 33
------ SEED: 34
------ SEED: 35
------ SEED: 36
------ SEED: 37
------ SEED: 38
------ SEED: 39
------ SEED: 40
------ SEED: 41
------ SEED: 42
------ SEED: 43
------ SEED: 44
------ SEED: 45
------ SEED: 46
------ SEED: 47
------ SEED: 48
------ SEED: 49
------ SEED: 50
------ SEED: 51
------ SEED: 52
------ SEED: 53
------ SEED: 54
------ SEED: 55
------ SEED: 56
------ SEED: 57
------ SEED: 58
------ SEED: 59
------ SEED: 60
------ SEED: 61
------ SEED: 62
------ SEED: 63
------ SEED: 64
------ SEED: 65
------ SEED: 66
------ SEED: 67
------ SEED: 68
------ SEED: 69
------ SEED: 70
------ SEED: 71
------ SEED: 72
------ S

------ SEED: 21
------ SEED: 22
------ SEED: 23
------ SEED: 24
------ SEED: 25
------ SEED: 26
------ SEED: 27
------ SEED: 28
------ SEED: 29
------ SEED: 30
------ SEED: 31
------ SEED: 32
------ SEED: 33
------ SEED: 34
------ SEED: 35
------ SEED: 36
------ SEED: 37
------ SEED: 38
------ SEED: 39
------ SEED: 40
------ SEED: 41
------ SEED: 42
------ SEED: 43
------ SEED: 44
------ SEED: 45
------ SEED: 46
------ SEED: 47
------ SEED: 48
------ SEED: 49
------ SEED: 50
------ SEED: 51
------ SEED: 52
------ SEED: 53
------ SEED: 54
------ SEED: 55
------ SEED: 56
------ SEED: 57
------ SEED: 58
------ SEED: 59
------ SEED: 60
------ SEED: 61
------ SEED: 62
------ SEED: 63
------ SEED: 64
------ SEED: 65
------ SEED: 66
------ SEED: 67
------ SEED: 68
------ SEED: 69
------ SEED: 70
------ SEED: 71
------ SEED: 72
------ SEED: 73
------ SEED: 74
------ SEED: 75
------ SEED: 76
------ SEED: 77
------ SEED: 78
------ SEED: 79
------ SEED: 80
------ SEED: 81
------ SEED: 82
------ S